In [1]:
import seaborn_image as isns
from pypatchify.pt import pt
from uda.datasets import MAndMs
from uda.metrics import dice_score, surface_dice
from ignite.utils import to_onehot
import torch

In [2]:
dataset = MAndMs("../config/dataset.yaml")
dataset.setup()

y_true = dataset.val_split.tensors[1]

Loading Testing data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.31s/it]


In [3]:
preds = y_true.clone()
targets = y_true.clone()

mask = (y_true > 0) * torch.Tensor([0.5, 0.8, 0.2, 1.0]).reshape(1, -1, 1, 1)
mask = torch.bernoulli(mask).bool()

preds[mask] = 0
preds[mask[:, [1, 2, 3, 0]]] = 1

preds.shape

torch.Size([48, 4, 256, 256])

In [4]:
for d in dice_score(preds, y_true):
    print(d)

tensor(0.6652)
tensor(0.2815)
tensor(0.5747)
tensor(6.3884e-12)


In [5]:
preds = preds.argmax(1)
targets = targets.argmax(1)
preds.shape

torch.Size([48, 256, 256])

In [6]:
if dataset.flatten:
    preds = preds.reshape(-1, dataset.imsize[0], *preds.shape[-2:])
    targets = targets.reshape(-1, dataset.imsize[0], *targets.shape[-2:])

preds = pt.unpatchify_from_batches(preds, dataset.imsize)
targets = pt.unpatchify_from_batches(targets, dataset.imsize)
preds.shape

torch.Size([4, 12, 256, 256])

In [7]:
preds = to_onehot(preds, 4)
preds.shape

torch.Size([4, 4, 12, 256, 256])

In [9]:
for d in dice_score(preds, targets):
    print(d)

tensor(0.6652)
tensor(0.2815)
tensor(0.5747)
tensor(6.3884e-12)


In [19]:
for d in surface_dice(preds, targets, dataset.val_spacings, tolerance_mm=1):
    print(d)

Computing surface dice:   0%|          | 0/16 [00:00<?, ?it/s]

tensor(0.1444)
tensor(0.3760)
tensor(0.7313)
tensor(0.0035)


In [20]:
preds.shape

torch.Size([4, 4, 12, 256, 256])

In [37]:
from itertools import chain
def remove_idx(t: torch.Tensor, idx: int, dim: int) -> torch.Tensor:
    assert idx < t.shape[dim]
    indices = torch.LongTensor(list(chain(range(idx), range(idx+1, t.shape[dim]))))
    return torch.index_select(t, dim, indices)

In [38]:
remove_idx(preds, 4, 1).shape

AssertionError: 